<a href="https://colab.research.google.com/github/ernestkung/NZCovidDocs/blob/master/NZ_Covid_19_Data_Dump_June_2020_Version_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Improved version for the June 2020 document dump. Builds on the May 2020 version.

Adds MySQL database to be able to track what was already downloaded last time. 

MD5 fingerprint is created for each document to detect matching documents.

Main source of documents: https://uniteforrecovery.govt.nz/updates-and-resources/legislation-and-key-documents/proactive-release/

In [ ]:
# Need the standard MySQL connector in Python
!pip install mysql-connector-python

     |████████████████████████████████| 14.8MB 289kB/s 


In [ ]:
# HERE'S THE PRODUCTION VERSION

from urllib import request
from bs4 import BeautifulSoup
import copy
import requests
import mysql.connector
from mysql.connector import errorcode
from mysql.connector.constants import ClientFlag
import datetime as dt
from dateutil import tz
import hashlib

def open_mysql_connection ():
  """Using MySQL settings to open connection to Google Cloud SQL service. Returns a MySQL connection object."""
  mysql_user = ""
  mysql_pwd = ""
  mysql_host = ""
  ssl_ca_file = ""
  ssl_key_file = ""
  ssl_cert_file = ""

  config = {
      'user': mysql_user,
      'password': mysql_pwd,
      'host': mysql_host,
      'client_flags': [ClientFlag.SSL],
      'ssl_ca': ssl_ca_file,
      'ssl_cert': ssl_cert_file,
      'ssl_key': ssl_key_file,
  }

  try:
    cnx = mysql.connector.connect(**config)
  except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
      return "Something is wrong with your user name or password"
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
      return "Database does not exist"
    else:
      return err
  else:
    return cnx

def convert_dt_to_mysql (
    input_datetime  
    ):
    """ Use this to convert a datetime value to UTC and into a format suitable for saving in MySQL. Requires datetime, dateutil tz to be imported. """ 
    utc_dt = input_datetime.astimezone(tz.gettz("Etc/UTC"))     
    mysql_dt = f"{utc_dt:%Y-%m-%d %H:%M:%S.%f}"
    return mysql_dt

download_directory = "/content/drive/My Drive/downloads2/wage_subsidy_and_leave_schemes/" # CHECK YOUR LOCAL DOWNLOAD DIRECTORY LOCATION
release_url = "https://uniteforrecovery.govt.nz/updates-and-resources/legislation-and-key-documents/proactive-release/wage-subsidy-and-leave-schemes/" # CHECK THIS LINK. CHANGE PER DOWNLOAD PAGE.
page_data = request.urlopen(release_url)
the_page = page_data.read()

chowder = BeautifulSoup(the_page, "html.parser")

bisque_documents = chowder.find_all("div", class_="content-element__content") # This is a list of each section of the page with the noted div + class tags

extracted_link = None
combined_link = None
extracted_title = None
extracted_dt = None
combined_date_time = None
extracted_type = None
current_section = 1

tz_auckland = tz.gettz("Pacific/Auckland") # This creates the timezone object to be used in a datetime Constructor.

for section in bisque_documents:  # Loops through each section of the page

  print ("Parsing occurence of the Div Class:", current_section)
  current_section = current_section + 1

  bisque_copy = copy.copy(section)

  bisque2_documents = bisque_copy.find_all("tr") # This is a list of each line with "td"
  print("Number of TR rows found:", len(bisque2_documents))

  for section2 in bisque2_documents:  # Loops through each instance of the "tr" tag

    bisque2_copy = copy.copy(section2)
    bisque3_documents = bisque2_copy.find_all("td")  

    print("Number of TD columns found:", len(bisque3_documents))
    
    if len(bisque3_documents) != 3: # Continue on to next loop because not having 3 TD columns means this is the header row.
      continue
    
    td_column_number = 1
    break_this_loop = 0

    for current_line in bisque3_documents:
      print ("Processing Column:", td_column_number)
      print (current_line)
     
      if td_column_number == 1:
        try:
          find_link = current_line.find("a")
          extracted_link = find_link.get("href")
        except:
          break_this_loop = 1
          continue
        else:
          pass
        
        start_of_link = "https://uniteforrecovery.govt.nz" # WATCH OUT FOR CHANGES IN THE DOMAIN NAME. IT ALREADY CHANGED ONCE FROM COVID19.GOVT.NZ.
        combined_link = start_of_link + extracted_link

      td_column_number = td_column_number + 1 
      # This loop section has been set up to accomodate future processing of the other available fields such as document title, document date, document type.
    
    if break_this_loop == 1:    # This was added because some pages use "td" for header rows instead of "th"
      continue
      
    print ("Link to Download:", combined_link)
    r = requests.get(combined_link)
    md5_r_file = hashlib.md5(r.content)   # The new innovation in this version is the generation of a MD5 fingerprint
    md5_r_file_digest = md5_r_file.hexdigest()
    print("MD5 of File on the Internet is: ", md5_r_file_digest)

    cnx = open_mysql_connection() # This calls a custom function that has the configuration info for MySQL service. Returns a MySQL connection object.
    cursor = cnx.cursor() 

    sql_code = f"""
    SELECT url_of_download FROM covid19docs.files 
    WHERE doc_hash = '{md5_r_file_digest}'
    LIMIT 1
    """
    # Matches the MD5 hash of the Internet file to the database to look for a match. If there isn't a match, the file will be downloaded.
    print (sql_code)

    cursor.execute(sql_code)

    content = cursor.fetchone()
    print (content)

    if content == None:
      utc_now = dt.datetime.now(tz.gettz("Etc/UTC")) 
      utc_now_mysql = convert_dt_to_mysql(utc_now)

      # Insert results to database. The database has a few more columns but they are not used right now.
      sql_code = f"""
      INSERT INTO covid19docs.files (doc_type_future, downloaded_dt_utc, url_of_download, added_by_user_id, doc_hash)
      VALUES ('1', '{utc_now_mysql}', '{combined_link}', '1', '{md5_r_file_digest}');
      """
      print (sql_code)

      try:
        cursor.execute(sql_code)
        
        file_name_start_index = combined_link.rfind("/")
        line_length = len(combined_link)
        file_name_iso = combined_link[(file_name_start_index+1):line_length]
        local_file_name = download_directory + file_name_iso

        try:
          with open(local_file_name, "xb") as f: 
            f.write(r.content)
            print("Saved file locally as:", local_file_name)
        except FileExistsError:
          print("File name already exists in Google Drive")
        else:
          pass      

      except Exception as e:
        cnx.rollback() # Rolls back the insert/updates to the database if anything goes wrong. 
        print(e)
      else:
        cnx.commit() # Commits the inserts/updates to the database. 

    if content != None:
      print ("The government previously released this file. It has the same MD5 signature as a file already in the MySQL database.")
 
      for x in content:
        print (x)
        
        if x != combined_link:
          sql_code = f"""
          UPDATE covid19docs.files 
          SET url_of_download_2 = '{x}'
          WHERE doc_hash = '{md5_r_file_digest}';
          """
          print (sql_code)
  
          try:
            cursor.execute(sql_code)
        
          except Exception as e:
            cnx.rollback() 
            print(e)
          else:
            cnx.commit() 

    cursor.close() 
    cnx.close() 

### ENDS ###
      

Parsing occurence of the Div Class: 1
Number of TR rows found: 15
Number of TD columns found: 0
Number of TD columns found: 3
Processing Column: 1
<td><a href="/assets/resources/proactive-release/COVID-19-How-self-isolating-workers-are-treated-under-workplace-law.pdf">COVID-19 - How self-isolating workers are treated under workplace law<span class="fileExt"> [PDF, 5.1 MB]</span></a></td>
Processing Column: 2
<td>4 March 2020</td>
Processing Column: 3
<td>Briefing</td>
Link to Download: https://uniteforrecovery.govt.nz/assets/resources/proactive-release/COVID-19-How-self-isolating-workers-are-treated-under-workplace-law.pdf
MD5 of File on the Internet is:  8bd7e37152514044a48a2b725fa8ad5c

    SELECT url_of_download FROM covid19docs.files 
    WHERE doc_hash = '8bd7e37152514044a48a2b725fa8ad5c'
    LIMIT 1
    
('https://uniteforrecovery.govt.nz/assets/resources/proactive-release/COVID-19-How-self-isolating-workers-are-treated-under-workplace-law.pdf',)
The government previously release